In [ ]:
# ============================================================
# 🌟 ADVANCED PROMPT ENGINEERING PLAYGROUND WITH FEW-SHOT CONTROL 🌟
# ============================================================

# 📚 This notebook demonstrates:
# - Prompt Engineering
# - Zero-Shot vs Few-Shot Prompting
# - System Prompting
# - Temperature control
# - Token understanding
# - Interactive Colab widgets
# - Side-by-side output comparison

# ============================================================

# Step 1: Install OpenAI SDK (if needed)
# ------------------------------------------------------------
# This installs the OpenAI Python library, needed to interact with OpenAI's models.
!pip install --upgrade openai > /dev/null 2>&1

# Step 2: Import necessary Python libraries
# ------------------------------------------------------------
from getpass import getpass            # Securely input your API key without displaying it
from openai import OpenAI                          # OpenAI's Python SDK
import ipywidgets as widgets           # For interactive widgets (dropdowns, sliders, buttons)
from IPython.display import display, Markdown # For displaying clean outputs

# Step 3: Collect API Key securely
# ------------------------------------------------------------
# Avoid hardcoding API keys for security reasons.
# getpass() prompts the user to input the key during runtime.

client = OpenAI(api_key=getpass("🔐 Enter your API key: "))

# Step 4: Define the core function to query the LLM
# ------------------------------------------------------------
# complete_prompt sends the prompt (and optional few-shot examples + system message) to the model.

def complete_prompt(prompt, system_message=None, temperature=0.7, model="gpt-4o-mini", max_tokens=500, examples=None):
    try:
        messages = []
        if system_message:
            messages.append({"role": "system", "content": system_message})

        if examples:
            for ex in examples:
                messages.append({"role": "user", "content": ex["user"]})
                messages.append({"role": "assistant", "content": ex["assistant"]})

        messages.append({"role": "user", "content": prompt})

        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens
        )

        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"❗ Error: {str(e)}"


# Step 5: Define the topic to explain/summarize
# ------------------------------------------------------------
topic = "The Earth revolves around the Sun, and this motion causes seasons to change due to the axial tilt."

# Step 6: Define prompt templates
# ------------------------------------------------------------
# Each template has:
# - A user instruction
# - An optional system message (to control tone/persona)
# - Optional few-shot examples

prompt_templates = {
    "Standard Summarization": {
        "prompt": f"Summarize this: {topic}",
        "system_message": None,
        "examples": [
            {"user": "Summarize this: Water boils at 100 degrees Celsius.",
             "assistant": "Water boils at 100°C under normal conditions."},
            {"user": "Summarize this: Photosynthesis lets plants use sunlight to make food.",
             "assistant": "Plants create food using sunlight via photosynthesis."}
        ]
    },
    "Friendly Teacher": {
        "prompt": f"Explain to a 5-year-old: {topic}",
        "system_message": "You are a friendly kindergarten teacher.",
        "examples": [
            {"user": "Explain to a 5-year-old: Why is the sky blue?",
             "assistant": "The sky looks blue because sunlight bounces off tiny air pieces."},
            {"user": "Explain to a 5-year-old: How does a tree grow?",
             "assistant": "Trees drink water from the ground and use sunshine to grow big!"}
        ]
    },
    "Technical Expert": {
        "prompt": f"Provide a detailed explanation: {topic}",
        "system_message": "You are a professional astrophysicist.",
        "examples": [
            {"user": "Provide a detailed explanation: What is gravitational lensing?",
             "assistant": "Gravitational lensing occurs when massive objects curve space-time, bending light from background sources."},
            {"user": "Provide a detailed explanation: What causes solar eclipses?",
             "assistant": "Solar eclipses occur when the Moon passes directly between the Earth and the Sun, blocking sunlight."}
        ]
    },
    "TL;DR Style": {
        "prompt": f"TL;DR: {topic}",
        "system_message": None,
        "examples": [
            {"user": "TL;DR: Gravity pulls things toward the Earth.",
             "assistant": "Gravity pulls stuff to Earth."},
            {"user": "TL;DR: Plants use sunlight and water to make food.",
             "assistant": "Plants make food using sun and water."}
        ]
    },
    "Question Reframing": {
        "prompt": f"What does it mean that Earth's tilt causes seasons?",
        "system_message": "Answer concisely and clearly.",
        "examples": [
            {"user": "Why does the ocean have tides?",
             "assistant": "Tides happen because the Moon’s gravity pulls on Earth’s oceans."},
            {"user": "Why do leaves fall off trees in autumn?",
             "assistant": "Trees lose leaves to save energy during winter."}
        ]
    }
}

# Step 7: Create Interactive Widgets
# ------------------------------------------------------------
style_dropdown = widgets.Dropdown(
    options=list(prompt_templates.keys()),
    value="Standard Summarization",
    description='Prompt Style:'
)

temperature_slider = widgets.FloatSlider(
    value=0.7,
    min=0.0,
    max=1.0,
    step=0.1,
    description='Temperature:',
    continuous_update=False
)

run_button = widgets.Button(
    description='Compare Outputs',
    button_style='success',
    tooltip='Click to run and compare Zero-Shot vs Few-Shot',
    icon='rocket'
)

output_area = widgets.Output()

# Step 8: Define Behavior to Run Both Zero-Shot and Few-Shot
# ------------------------------------------------------------
def on_button_click(b):
    output_area.clear_output(wait=True)  # ✅ Clear old output immediately

    selected_style = style_dropdown.value
    selected_temp = temperature_slider.value
    prompt_info = prompt_templates[selected_style]

    try:
        # Start model calls
        zero_shot_output = complete_prompt(
            prompt=prompt_info['prompt'],
            system_message=prompt_info['system_message'],
            examples=None,
            temperature=selected_temp
        )

        few_shot_output = complete_prompt(
            prompt=prompt_info['prompt'],
            system_message=prompt_info['system_message'],
            examples=prompt_info['examples'],
            temperature=selected_temp
        )

        # Now: All display inside one fresh output_area
        with output_area:
            display(Markdown(f"# 📝 Prompt Style: **{selected_style}**"))
            display(Markdown(f"🌡️ **Temperature:** `{selected_temp}`"))
            display(Markdown("---"))
            display(Markdown("### 🎯 **Zero-Shot Output**"))
            display(Markdown(f"```text\n{zero_shot_output}\n```"))
            display(Markdown("---"))
            display(Markdown("### 🚀 **Few-Shot Output**"))
            display(Markdown(f"```text\n{few_shot_output}\n```"))

    except Exception as e:
        # Catch errors
        with output_area:
            display(Markdown(f"❗ **An error occurred:** `{str(e)}`"))

# Step 9: Display the Interface
run_button.on_click(on_button_click)
# ------------------------------------------------------------
display(style_dropdown, temperature_slider, run_button, output_area)